<a href="https://colab.research.google.com/github/5ury4pr454th/building-scene-classification/blob/main/Detect_and_Save_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Files

In [1]:
#@title
# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# for mounting drive and basic file navigation
import os
from google.colab import drive

# for saving csv
import pandas as pd

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# for additional dictionary tools
from collections import Counter

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

# mounting drive for using stored data
drive.mount('/content/gdrive')

2.17.0
The following GPU devices are available: /device:GPU:0
Mounted at /content/gdrive


## Defining some functions for image preprocessing

In [2]:
#@title
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.axis('off')
    plt.imshow(image)
    plt.show()


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOs)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color, font, thickness=4, display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)],
              width=thickness, fill=color)

    display_str_heights = [font.getbbox(ds)[3] - font.getbbox(ds)[1] for ds in display_str_list]
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = bottom + total_display_str_height

    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getbbox(display_str)[2], font.getbbox(display_str)[3] - font.getbbox(display_str)[1]
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str, fill="black", font=font)
        text_bottom -= text_height + 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=15, min_score=0.2):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype("/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/indian-street-scenes/LiberationSansNarrow-Regular.ttf", 25)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"), int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, font, display_str_list=[display_str])

    return np.array(image_pil)


def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_detector(detector, path, show=True):
    img = load_img(path)

    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()

    result = {key: value.numpy() for key, value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time - start_time)

    image_with_boxes = draw_boxes(img.numpy(), result["detection_boxes"], result["detection_class_entities"], result["detection_scores"])
    if show:
        display_image(image_with_boxes)

    return result["detection_boxes"], result["detection_class_entities"], result["detection_scores"]



def resize_and_save(image_path, final_path, width, height):
  pil_image = Image.open(image_path)
  pil_image = ImageOps.fit(pil_image, (width, height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(final_path, format="JPEG", quality=90)
  pass

def crop_bounding_box(img, bounding_box):
  ymin, xmin, ymax, xmax = tuple(bounding_box)
  return img.crop((xmin, ymin, xmax, ymax))

def fit_and_save(img, final_path, width, height):
    cropped_img = ImageOps.fit(img, (width, height), Image.LANCZOS)
    cropped_img_rgb = cropped_img.convert("RGB")
    cropped_img_rgb.save(final_path, format="JPEG", quality=90)
    pass


def do_operations(filename):

  # open image file
  img = Image.open(filename)

  # rescale image
  img = ImageOps.fit(img, (resized_img_size, resized_img_size), Image.LANCZOS)

  # convert to RGB if not
  img = img.convert("RGB")

  np_img = np.array(img)

  return np_img

## Preprocessing

Processando Imagens Teste

In [3]:
#Modificando o tamanho das imagens
original_image_path = "/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/test"

# Caminho onde as imagens redimensionadas serão salvas
resized_image_path = "/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/test-resized"


resized_street_image_dims_width = 700
resized_street_image_dims_height = 450


for root, dirs, files in os.walk(original_image_path):
    for file in files:
        file_path = os.path.join(root, file)
        relative_path = os.path.relpath(root, original_image_path)
        save_dir = os.path.join(resized_image_path, relative_path)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, file)
        resize_and_save(file_path, save_path, resized_street_image_dims_width, resized_street_image_dims_height)

Processando Imagens Treinamento

In [ ]:
#Modificando o tamanho das imagens
original_image_path = "/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/train"

# Caminho onde as imagens redimensionadas serão salvas
resized_image_path = "/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/train-resized"


resized_street_image_dims_width = 700
resized_street_image_dims_height = 450


for root, dirs, files in os.walk(original_image_path):
    for file in files:
        file_path = os.path.join(root, file)
        relative_path = os.path.relpath(root, original_image_path)
        save_dir = os.path.join(resized_image_path, relative_path)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, file)
        resize_and_save(file_path, save_path, resized_street_image_dims_width, resized_street_image_dims_height)

Classes para funcionar de acordo com o dataset de teste

In [4]:
#classes para funcionar de acordo com o dataset de teste
classes = ["residential", "commercial","others","industrial"]

In [ ]:
#classes para funcionar de acordo com o dataset de treinamento
classes = ["residential", "mixed", "commercial","others","industrial"]

## Detection using pretrained model

In [5]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

In [6]:
resized_img_size = 500
cropped_dims = 800
frequency_dict = dict()
predicted_dict = {}
resized_street_image_dims = 800
instances_folder = "/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/results_test"

resized_images_path = '/content/gdrive/MyDrive/Colab Notebooks/TCC/Scripts/imagens_bd/test-resized'

In [7]:
expansions = [resized_street_image_dims_height, resized_street_image_dims_width, resized_street_image_dims_height, resized_street_image_dims_width]

In [8]:
for bclass in classes:

  for i in os.listdir(resized_images_path +'/'+ bclass + '/'):

    predicted_dict[bclass +'/' + i] = []
    bounding_objects = []

    # for each image, run the detector and get the bounding boxes and corresponding
    img = Image.open(resized_images_path + '/'+ bclass + '/' + i)
    bounding_boxes, raw_bounding_classes, class_scores = run_detector(detector, resized_images_path + '/' + bclass + '/' + i, show=True)

    # for each of the objects detected, check if house, skyscraper or building, crop according to priority, and save them to the cropped_folder
    for index, i3 in enumerate(raw_bounding_classes):
      i1 = str(i3)[2:-1].lower()
      if i1 in ['house', 'building', 'skyscraper', 'tower'] and class_scores[index] > 0.3:
        final_path = instances_folder + "/" + bclass + '/' + str(index) + '_' + i
        expanded_boxes = np.asarray(tuple(expansions)) * np.asarray(bounding_boxes[index])
        house_bound = crop_bounding_box(img, expanded_boxes)
        house_bound = fit_and_save(house_bound, final_path, 500, 700)

    for objects in range(len(raw_bounding_classes)):
      if class_scores[objects] >= 0.4:
        bounding_objects.append(str(raw_bounding_classes[objects])[2:-1].lower())

    # record the number of objects detected
    frequency_dict[bclass +'/' + i] = Counter(bounding_objects)